# Air Quality Assessment in the Arctic
In this notebook we analyze the air quality in the arctic area using the data from ground stations located in European arctic

* Grundartangi Groef, Iceland (EEA st.52149) 
* Kopavogur Dalsmari, Iceland (EEA st.52109)
* Oulu Pyykoesjaervi, Finland (EEA st. 15557)
* Oulun Keskusta 2, Finland (EEA st. 15609)
* Tromso, Norway (EEA st. 62993) 
* Muonio Sammaltunturi, Finland (FMI st. 101983) 

The datasets are provided by the [Finnish Meteorological Institute (FMI)](https://en.ilmatieteenlaitos.fi/download-observations) and the [European Environmental Agency (EEA)](https://discomap.eea.europa.eu/map/fme/AirQualityExport.htm). The aim of the project is to build a model to predict the state of environmental variables such as the PM10. The prediction will be compared with that from the [CAMS service](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts). The prediction provided by the CAMS are the result of an ensemble of 9 air quality numerical models. An air quality model integrates a dynamical model of the atmosphere with the chemical-physical equations of the air components. The CAMS numerical models use as input the data assimilated from a network of ground stations and satellite sensors and return as output a grid that represents the horizontal and vertical distribution of the air quality variables with a prediction of their future state.      


In [4]:
import pandas as pd
#import geopandas as gpd
#from shapely.geometry import Point, LineString, Polygon
import matplotlib.pyplot as plt

## The datasets
Five of the 6 datasets can be downloaded from the EEA website by selecting the name of the country, the polluttant (PM10), and the years. Each station has a code number that can be used to filter out the files to be dowloaded. The dataset of the station managed by the FMI, Muonio Sammaltunturi, can be downloaded from the FMI website by selecting the hourly Air Quality observations of the Muonio Sammaltunturi meteorological institute for the inhalable particles <10 $\mu m$ and the time period.    


## References
* [Fazzini et al. - Forecasting PM10 Levels Using Machine Learning Models in the Arctic: A Comparative Study](https://www.mdpi.com/2072-4292/15/13/3348)  
* [ECMWF - CAMS Regional: European air quality analysis and forecast data documentation](https://confluence.ecmwf.int/display/CKB/CAMS+Regional%3A+European+air+quality+analysis+and+forecast+data+documentation)